# CHANGE DETECTION

## Instalar
Instalar geemap y eeconvert en Google Colab.

In [ ]:
!pip install geemap
!pip install eeconvert

## Importar

- Google Earth Engine y utilidades

In [1]:
import ee, eeconvert, geemap.eefolium as geemap

- Manejo de (Geo) Data Frames

In [2]:
import geopandas as gpd, pandas as pd

- Manejo de Arrays

In [3]:
import numpy as np

## Iniciar GEE
- Initialization

In [4]:
ee.Initialize()

## Datos

Directorio donde se encuentra el shapefile.

In [6]:
import os,glob
g = os.getcwd()
g
path = 'D:\\repos\\CIAT\\IDB_PROJECT'
os.chdir(path)

Lectura del shapefile y cambio del sistema de referencia.

In [7]:
points = gpd.read_file("Data_IDB/new/Zona_influencia_DR.shp").to_crs("EPSG:4326")

Asignación de un identificador único para cada registro.

In [8]:
points['ID'] = points['cuest_id_b'].astype(str) + points['parcela'].astype(str)

Obtener sólo los registros corregidos.

In [9]:
points = points[points['corregido2'].str.contains("ok")][['ID','patca','geometry']]

Convertir a Feature Collection.

In [10]:
eepoints = eeconvert.gdfToFc(points)

## Preprocesamiento de Imágenes

Nombres en para bandas en común entre Landsat 7 y Landsat 8. Esto con el fin de crear una única colección de imágenes con ambos datasets.

In [11]:
bandNames = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']

Bandas correspondientes para cada satélite.

In [12]:
L7Bands = ['B1','B2','B3','B4','B5','B7']
L8Bands = ['B2','B3','B4','B5','B6','B7']

Lectura de cada uno de los datasets. Se realizan los filtros correspondientes por ubicación, fecha y se seleccionan únicamente las bandas en común (también se renombran).

In [13]:
L7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterBounds(eepoints).filterDate('2011-01-01','2020-01-01').select(L7Bands,bandNames)
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(eepoints).filterDate('2011-01-01','2020-01-01').select(L8Bands,bandNames)

Unir ambas colecciones en una sola y ordenar por fecha.

In [14]:
L78 = L7.merge(L8).sort('system:time_start')

## Detección de Cambios

### Algoritmo Continuous Change Detection and Classification
Este algoritmo recibe una colección de imágenes, utiliza todas las bandas para ajustar funciones armónicas y encontrar fechas de quiebre cuando las funciones cambian (el algoritmo funciona a base de pixel).

> **PUEDE HABER MÁS DE UNA FECHA DE QUIEBRE**

- Se utilizaron todas las bandas para el algoritmo (el algoritmo da la opción de usar sólo las bandas seleccionadas, así que este parámetro puede cambiarse a gusto).
- Se utilizaron las bandas VERDE y SWIR2 para enmascarar nubes y sombras usando el algoritmo TMASK-
- El formato de fecha de salida es MILISEGUNDOS.



In [15]:
CCDC = ee.Algorithms.TemporalSegmentation.Ccdc(collection = L78,tmaskBands = ['GREEN','SWIR2'],dateFormat = 2)

El algoritmo encuentra segmentos y guarda las fechas de acuerdo a segmentos:

- tStart: Inicio de segmento.
- tEnd: Fin de segmento.
- tBreak: Punto de quiebre (inicio del siguiente segmento: siguiente _tStart_. El fin de un segmento _tEnd_ no es necesariamente el inicio de otro segmento _tStart_ y por lo tanto no necesariamente un punto de quiebre _tBreak_).
- numObs: Observaciones del segmento.

Estos valores se extraen por cada registro del Feature Collection. Al ser polígonos, sólo nos interesa un valor (ee.Reducer.first()).

In [16]:
eepointsCCDC = CCDC.select(['tStart','tEnd','tBreak','numObs']).reduceRegions(collection = eepoints,reducer = ee.Reducer.first(),scale = 30,tileScale = 16)

Convertir valores extraídos a un Data Frame.

In [17]:
pointsCCDC = eeconvert.fcToGdf(eepointsCCDC)

C:\Users\fmontenegro\Miniconda3\envs\GEE\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Los valores se guardan en listas (el tamaño de la lista es proporcional al número de segmentos encontrados). Más de un segmento significa que un cambio fue encontrado. Estas listas son explotadas y apiladas en la misma columna.

In [18]:
tStart = pointsCCDC['tStart'].explode()
tEnd = pointsCCDC['tEnd'].explode()
tBreak = pointsCCDC['tBreak'].explode()
numObs = pointsCCDC['numObs'].explode()

Ya que las listas fueron explotadas y apiladas en la misma columna, los índices de los registros que se apilaron se utilizan para replicar los registros del data frame. Si una lista tiene dos segmentos para un registro, este debe ahora aparecer dos veces.

In [19]:
pointsExploded = pointsCCDC.loc[numObs.index,['ID','patca','geometry']]

Se añaden los valores de las listas explotadas al data frame donde se replicaron los registros. En el caso de los datos de fecha, estos son formateados de acuerdo a sus unidades de MILISEGUNDOS.

In [20]:
pointsExploded['tStart'] = pd.to_datetime(tStart,unit = 'ms')
pointsExploded['tEnd'] = pd.to_datetime(tEnd,unit = 'ms')
pointsExploded['tBreak'] = pd.to_datetime(tBreak,unit = 'ms')

En el caso del número de observaciones no se realiza ningún cambio.

In [21]:
pointsExploded['numObs'] = numObs

Cuando sólo existe un segmento, no existe (generalmente) punto de quiebre y su valor de MILISEGUNDOS es configurado en cero, lo que equivale a una fecha de 1970 (esto también ocurre a veces en el segmento final cuando hay más de un segmento). Así que podemos cambiar estos valores para evitar confusiones.

In [22]:
pointsExploded.loc[pointsExploded['tBreak'] < '2011-01-01','tBreak'] = np.nan

## Ejemplos

El registro con ID 386311 sólo tiene un segmento de 131 imágenes que inicia en 2011-01-01 y termina en 20019-12-17. No se ecnontró ningún punto de quiebre o cambio de cobertura en las imágenes.

In [23]:
pointsExploded[pointsExploded['ID'] == '386311']

,ID,patca,geometry,tStart,tEnd,tBreak,numObs
0,386311,1,"POLYGON ((-71.70753 18.89259, -71.70756 18.892...",2011-01-01 15:01:56.470,2019-12-17 15:09:03.062,NaT,131


El registro con ID 396011 tiene 4 segmentos con 47, 42, 31 y 6 imágenes respectivamente. El primer segmento inicia en 2011-01-01 y termina en 2015-03-25. El segundo segmento inicia en 2015-04-10 y termina en 2018-01-28. Y así sucesivamente.

In [24]:
pointsExploded[pointsExploded['ID'] == '396011']

,ID,patca,geometry,tStart,tEnd,tBreak,numObs
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2011-01-01 15:01:56.470,2015-03-25 15:08:16.830,2015-04-10 15:08:08.210,47
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2015-04-10 15:08:08.210,2018-01-28 15:08:41.000,2018-02-13 15:08:34.810,42
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2018-02-13 15:08:34.810,2019-09-12 15:09:01.304,2019-09-28 15:09:06.465,31
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2019-09-28 15:09:06.465,2019-12-17 15:09:03.062,NaT,6


In [25]:
pointsExploded.head()

,ID,patca,geometry,tStart,tEnd,tBreak,numObs
0,386311,1,"POLYGON ((-71.70753 18.89259, -71.70756 18.892...",2011-01-01 15:01:56.470,2019-12-17 15:09:03.062,NaT,131
1,386512,1,"POLYGON ((-71.68327 18.70962, -71.68331 18.709...",2011-01-17 15:02:04.340,2019-12-17 15:09:03.062,NaT,128
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2011-01-01 15:01:56.470,2015-03-25 15:08:16.830,2015-04-10 15:08:08.210,47
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2015-04-10 15:08:08.210,2018-01-28 15:08:41.000,2018-02-13 15:08:34.810,42
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2018-02-13 15:08:34.810,2019-09-12 15:09:01.304,2019-09-28 15:09:06.465,31
